# `qubit_discovery` Transmon Optimization

Here, we use `qubit_discovery` to optimize an asymmetric transmon for a specific target frequency $f_{01}^*$ and long $T_2$ time. The point of this notebook is to show an entire workflow start-to-end; see the other tutorial notebooks for more details about the different functions used.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import qubit_discovery as qd
import SQcircuit as sq

: 

1. Initialize a sampler to randomly sample circuits from allowed element ranges.

In [2]:
# Turn on optimization
sq.set_optim_mode(True)

# Construct circuit sampler
sampler = qd.optimization.CircuitSampler(
    capacitor_range=[1e-15, 12e-12],    # in F
    inductor_range=[1e-15, 5e-6],       # in H
    junction_range=[1e9, 100e9],        # in Hz
    flux_range=[0, 2 * np.pi]   
)

2. Construct loss function, using some built-in and some custom metrics. We'll optimize for
- A target qubit frequency of 6.5 GHz (custom metric)
- A long $T_2$ time.
- Low sensitivity to external flux and charge

In [3]:
# Construct a custom metric function for target frequency
TARGET_FREQUENCY = 6.5

def target_freq_loss(circuit):
    circuit_freq = circuit.efreqs[1] - circuit.efreqs[0]

    loss = ((circuit_freq - TARGET_FREQUENCY)/TARGET_FREQUENCY)**2
    return loss, circuit_freq

qd.losses.add_to_metrics('target_frequency', target_freq_loss)


# Build total loss function 
my_loss_function = qd.losses.build_loss_function(
    use_losses={ # setting all weights equal for this example
        'target_frequency': 1.0,
        't2': 1.0,
        'charge_sensitivity': 1.0,
        'flux_sensitivity': 1.0
    },
    use_metrics=['t1', 't_phi'] # interesting to track T1 and T_phi
)

3. Perform the optimization. Because gradient descent can get stuck in local minima, we run optimization 10 times with different random initializations.

In [4]:
optimized_circuits = []
optim_records = []

for i in range(10):
    circuit = sampler.sample_circuit_code('JJ')
    final_circuits, params, optim_results = qd.optimization.run_BFGS(
        circuit = circuit,
        loss_metric_function  = my_loss_function,
        max_iter = 30,
        total_trunc_num = 50,
        bounds = sampler.bounds,
    )
    optimized_circuits.append[final_circuits]
    optim_records.append(optim_results)

set_trunc_nums called
diag called
diag called
diag called
diag called
diag called


TypeError: DecRateCharge.forward() takes 3 positional arguments but 4 were given

In [5]:
cr = sampler.sample_circuit_code('JJ')

In [8]:
cr.set_trunc_nums([120,])
cr.diag(10)
cr.dec_rate('charge', (0, 1))

set_trunc_nums called
diag called


TypeError: DecRateCharge.forward() takes 3 positional arguments but 4 were given

In [6]:
import torch
torch.__version__

'1.12.0'

If